---
# makemore Part 2: MLP
---

An implementation of [makemore library](https://github.com/karpathy/makemore) by Mr. Karpathy as taught in his [video](https://youtu.be/TCH_1BHY58I). makemore is a model that makes more of things that you give it. It is a character level language model, meaning that it treats example words as sequences of individual characters.


In our first approach to language modeling we used a bigram character-level model. We started off with a counting approach in which we recorded the frequency of bigrams present in the training data in a table. There are a total of $26$ letters in the English alphabet, and we include a special token used to designate the beginning and ending of a word. This gives us a table of shape $27 \times 27$. 
The first row of the table consists of the counts for all the starting characters. The columns of the table give the first character in the bigram, and the rows give the second character. 

After normalizing the rows, we were able to sample from the model by first drawing a character from the first row. Then we would get the row corresponding to the second character distribution given the first character. From this row, we would draw the second character in the bigram, which would then become the first character in the next bigram. This process would continue until the special token was selected.

This approach is alright, but it does not scale well. For example, if we simply want to consider single-character $1$-grams, then we would have a probability distrubtion over $27$ tokens. For a bigram, we considered a table of $27 \times 27$ two-letter sequences. For a trigram, we would need to store counts in a $27 \times 27 \times 27$ array. Thus, the number of sequences we would need to store for an n-gram scales as $27^n$.


The second gradient-based approach is much more scalable. We created a single-layer neural network with $27$ fully connected neurons with a collective softmax activation (cool graphic that I made using [this website](http://alexlenail.me/NN-SVG/index.html)).
The input to the network was a $27$-dimensional one-hot-encoded vector that represents the first character in a bigram sequence. The output was a $27$-dimensional vector that represents the probability distribution of the next character in the sequence. It turned out that our weight matrix, `W`, which represents the connections between inputs and neurons, eventually converged to the bigram frequency table with training (actually, log counts). This helped build up our intuition about what the matrix multiplication between the input vector and weight matrix was doing. Because the input vector is one-hot-encoded at a particular index $i$, `logits = xenc @ W` leads to the extraction of the $i$ th row of `W`. As `W` is the bigram count table, the extracted row gives the frequencies of successive characters to the input charcter. When normalized, it is our probability distribution.

<div style="display: flex;">
    <img src="img/bigram_count_table.png" alt="bigram count table" width="900" style="margin-right: -300px;">
    <img src="img/horizontal_nn.svg" alt="neural net" width="1200">
</div>



---
## 1) Basic idea of an MLP language model


The MLP model that we are building is based on the word-level language model in [this paper](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf). In their approach, they consider a large vocabulary of thousands of words that serve a similar purpose to the 27 characters in our character-level model. For our model, we could get away with one-hot-encoding each input character because we only had a total of 27, meaning that our feature space consisted of 27 dimensions. However, because one-hot-encoding produces vectors of the size of the number of possible tokens, this encoding is expensive for large numbers of tokens. Thus, the authors of the paper choose to use a neural network to learn lower-dimensional embedding vectors for each word (tens of features instead of thousands). They then use an approach for modeling that is very similar to what we have done: train a model to predict the next word given a sequence of words, optimized by maximizing the log-likelihood of the training data.

The process of embedding is a technique that aims to reduce the dimensionality of a particular feature space while retaining information about the distribution of points within it. In the paper, the embeddings of each words are randomly initialized in the lower-dimensional space, and are tuned through backpropagation. During training, these vectors move throughout the space. Words with similar semantic and/or syntactic meanings should end up near each other. Words with different semantic meanings should end up in different places. 

The clustering of similar words in the embedding space helps to generalize the model to deal with out-of-distribution inputs during test time. So when an input phrase that the model did not see during training appears, it should be able to predict the next word based on the proximity of the input sequence of words to known words in the embedding space.For example, the embeddings might have been tuned to learn that the words "a" and "the" have similar meanings and are often used interchangeably, or that "dogs" and "cats" are both animals.

Suppose the model was trained on the phrase "A dog was running in a room", and is then prompted to predict the next word in the sequence "The cat is running in a", which is not explicitly in the training data. Because the two phrases are similar enough, it should be able to come up with a reasonable word to complete the sentence. So we can transfer knowledge through word proximity in the embedding space in order to generalize to novel scenarios.

The paper uses the neural network pictured below. In this particular example, the network uses a three-word context to figure out the fourth word in the sequence. The words are fed into the neural network by their integer-valued index. Each of the roughly 17,000 possible words is given an index, 0 to 16,999. This index is used to pull a lower-dimensional representation of the word from a look-up table. Let's say that we want features with 30 dimensions, then the look-up table would be a $17,000 \times 30$ matrix. If we stuck to one-hot-encodings, then we would be dealing with a $17,000 \times 17,000$ matrix (all encodings stacked on top of each other). So this look-up table stores the embedding vectors for all words in the vocabulary, and because it is connected to the rest of the net, it learns these embeddings through backpropagation. Getting back to the example, if we pull 3 30-dimensional vectors (3-word context length), then our input layer has a total of 90 neurons. Then hidden layer in the middle has a variable number of neurons (a hyperparameter) that are fully connected to the input layer, and it has a tanh activation function. The output layer has 17,000 neurons with a collective softmax activation to create a probability distribution over the 17,000 possible words (logits). This is the most computationally expensive layer because it is fully connected to the hidden layer. If the hidden layer has $n$ neurons, then there are a total of $17,000 \times n$ parameters stored in the weight matrix. The weight matrix between the inputs and hidden layer is shape $n \times 90$. The final output is an index drawn from the probability distribution.


<img src="img/bengio_nn.png" alt="neural net" width="400">


---
## 2) Rebuilding the training set

First, we need to create our training set of names.

---

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read the words
import os
data_path = os.path.join("data", "names.txt")
with open (data_path, "r") as f:
    words = f.read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
print("data points:", len(words))
print("longest word:", max(len(w) for w in words))
print("shortest word:", min(len(w) for w in words))
print("first 10 names:", words[:10])

data points: 32033
longest word: 15
shortest word: 2
first 10 names: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [9]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set("".join(words))))   # sort all of the unique chars in the list of words
stoi = {s:i+1 for i,s in enumerate(chars)}  # map char to ints, add an index for a special token
stoi["."] = 0                               # insert special token to represent start and end of words
itos = {i:s for s,i in stoi.items()}        # map int index to char

In [5]:
print("stoi:", stoi)
print("itos:", itos)

stoi: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
itos: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# context length: how many characters do we take to predict the next one?
block_size = 3

# X is nn input, Y is label
X, Y = [], []

for w in words[:5]:
    print(w)
    
    # for each word, start with the 0 index, which maps
    # to the special token to indicate the start
    # we need at least block_size many to begin to predict the (padded context)
    # first letter, this will ultimately be a rolling window of context of
    # length block_size that stores the int encoding of each token
    # in the word e.g., block_size = 3: 
    # "context ---> true_output": "... ---> e", "..e ---> m", ...
    context = [0] * block_size

    # iterate over chars in words with the special
    # token appended to the end 
    for ch in w + ".":
        idx = stoi[ch]     # get the index of the current character
        X.append(context)  # add the context, the input to the network
        Y.append(idx)      # add the true output given the context

        # for each int in the context, map it to char and print the true output
        print("".join(itos[i] for i in context), "--->", itos[idx])

        # move the context window over by one
        # context[1:] gives the list of all but the first element
        # and + [idx] appends idx, the next char encoding, to the context
        context = context[1:] + [idx] # crop and append
    
# convert python lists to torch tensors
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


The training set `X` is a $N \times D$ matrix, and the corresponding vector of labels `Y` has $N$ elements.  $N$ is the number of samples, and $D$ is the number of features. In our case, $D$ is equal to the `block_size` -- the context length for predicting the next character. $N$ is a little more tricky to figure out, but we just need to remember what our goal is. We want to predict the next character given a sequence of previous characters, so our labels should simply be the integer-encoded characters in the training set (plus the special ending token). Thus, $N$ is equal to the total number of letters in the training set plus the number of words (each word has one ending token).

In [7]:
# the columns of X are the 
# the rows of X 
print("X.shape:", X.shape)
print("X.dtype:", X.dtype)
print(X[:4])
print("Y.shape:", Y.shape)
print("Y.dtype:", Y.dtype)
print(Y[:4])

X.shape: torch.Size([32, 3])
X.dtype: torch.int64
tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13]])
Y.shape: torch.Size([32])
Y.dtype: torch.int64
tensor([ 5, 13, 13,  1])


---
## 3) Implementing the embedding lookup table

In the paper, they reduced the dimensionality from 17,000 to 30. We only have 27 characters, so we will try an embedding with two dimensions to start with. 


---

We start by initializing a random array `C` with $27$ rows representing each possible character and $2$ columns representing each feature. To get the embedding vector for the character with integer index $i$, we can simply index the lookup table `C[i]`. We can convert this into a matrix multiplication by one-hot-encoding the character with index $i$ using `F.one_hot(torch.tensor(5), num_classes=27).float()`, where we cast from a long to a float. Then `F.one_hot(torch.tensor(5), num_classes=27).float() @ C ` has shape $(1 \times 27) @ (27 \times 2) = (1 \times 2)$ (the one-hot-encoded vector is broadcasted from 27 to 1x27). The one-hot-encoded vector multiplies each column of `C`. Because most of the vector's entries are equal to zero, only the $i \text{th}$ row of `C` is extracted. For a more in-depth explanation, see section 12.1 of [Part 1](part_1_bigrams.ipynb). 

So we can think of this embedding as an integer indexing into a lookup table, or as being the first layer of a neural network. This layer has no non-linear activation functions, and has a weight matrix given by `C`. For our purposes, it is slower to one-hot-encode the integer first, so we will take the indexing approach.

In [13]:
C = torch.randn((27, 2))

In [17]:
C[5]

tensor([0.2053, 1.0429])

In [20]:
print(F.one_hot(torch.tensor(5), num_classes=27))
print(F.one_hot(torch.tensor(5), num_classes=27).shape)
print(F.one_hot(torch.tensor(5), num_classes=27).dtype)

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])
torch.Size([27])
torch.int64


In [16]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([0.2053, 1.0429])

We can index multiple values within a `torch.tensor` at once using a different types of containers (list and tensor shown).

In [21]:
C[[5, 6, 7]]

tensor([[ 0.2053,  1.0429],
        [-0.2128,  1.4304],
        [-0.5183, -0.7712]])

In [22]:
C[torch.tensor([5, 6, 7])]

tensor([[ 0.2053,  1.0429],
        [-0.2128,  1.4304],
        [-0.5183, -0.7712]])

We can also perform multi-dimensional indexing. `X` has two dimensions, `X.shape: torch.Size([32, 3])`. `C[X]` retrieves a two-dimensional embedding vector for each of the $32 \times 3$ vectors in `X`, `C[X].shape: torch.Size([32, 3, 2])`.

In [24]:
C[X].shape

torch.Size([32, 3, 2])

In [27]:
print(X[13, 2])
print(C[X][13, 2])
print(C[1])

tensor(1)
tensor([-1.2317, -0.3320])
tensor([-1.2317, -0.3320])


So we can use this to embed all of our vectors at once

In [ ]:
emb = C[X]

---
## 4) Implementing the hidden layer

---